ETL STEAM_GAMES

In [1]:
#Importamos las librerías a utilizar
import pandas as pd
import json
import ast
import csv

In [2]:
#Leemos y convertimos el archivo json a dataframe
with open(r"C:\Users\jotad\OneDrive\Documentos\Sebastian\PROYECTO\output_steam_games.json", 'rt', encoding='utf-8') as f:
    data = f.readlines()

In [3]:
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"
df_games = pd.read_json(data_json_str)

In [4]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [5]:
#Eliminamos las columnas que no se necesitan para el procesamiento
#df_games = df_games.drop(columns = ['user_id','discount_price','items','steam_id','items_count'])

In [6]:
#Eliminamos los registros con valor nulo en todos los campos
df_games.dropna(how='all', inplace=True)

In [7]:
#Convertimos en 0 los registros que no tienen id
df_games[id]=df_games['id'].fillna(0).astype(int)
df_games[id]=df_games['id'].fillna(0)

In [8]:
#Generación de la variable año desde release_date
#Se da formato fecha a los valores de la columna release_date, quedando como nulos los que no tienen el formato
df_games['release_date']=pd.to_datetime(df_games['release_date'], errors='coerce')

#Se asegura que los datos cumplan con el formato AAAA-MM-DD
df_games['release_date']=pd.to_datetime(df_games['release_date']).dt.strftime('%Y-%m-%d')

#Se identifican los valores nulos de la columna release_date y se eliminan
df_games['release_date'].isnull().sum()
df_games = df_games.dropna(subset=['release_date'])

#Se crea la variable year
df_games['release_date']=pd.to_datetime(df_games['release_date'])
df_games['year']=df_games['release_date'].dt.year

#Se verifican los cambios
df_games[['release_date','year']].tail(15)

,release_date,year
120428,2018-01-03,2018
120429,2018-01-03,2018
120430,2016-11-19,2016
120431,2018-01-03,2018
120432,2018-01-03,2018
120434,2018-01-03,2018
120435,2018-10-01,2018
120436,2018-01-04,2018
120437,2018-01-04,2018
120438,2018-01-04,2018


In [9]:
#Transformamos la columna price
#Reemplazamos los valores free a 0
df_games['price'] = df_games['price'].replace(['Free to Play','Free Demo','Free HITMANâ„¢ Holiday Pack','Play the Demo','Free Mod','Play WARMACHINE: Tactics Demo','Free To Play','Free'], 0)

In [10]:
#Reemplazamos los registros de price que noi tiene relación con el precio por NaN
df_games['price'] = df_games['price'].replace(['Install Theme','Third-party','Starting at $','Starting at $','Play Now','Starting at $499.00','Starting at $449.00'], 'NaN')

In [11]:
#Eliminamos los regsitros NaN de price
df_games.dropna(subset=['price'])

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,2376272425360,year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,761140.0,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0,0.0,643980.0,Secret Level SRL,643980.0,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0,0.0,670290.0,Poolians.com,670290.0,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,767400.0,2017
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd,772540.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,0.0,745400.0,Bidoniera Games,745400.0,2018
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS""",773640.0,2018
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada,733530.0,2018
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,610660.0,2018


In [12]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29783 entries, 88310 to 120443
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   publisher      23793 non-null  object        
 1   genres         28549 non-null  object        
 2   app_name       29782 non-null  object        
 3   title          29782 non-null  object        
 4   url            29783 non-null  object        
 5   release_date   29783 non-null  datetime64[ns]
 6   tags           29622 non-null  object        
 7   reviews_url    29782 non-null  object        
 8   specs          29114 non-null  object        
 9   price          28782 non-null  object        
 10  early_access   29783 non-null  float64       
 11  id             29782 non-null  float64       
 12  developer      28533 non-null  object        
 13  2376272425360  29783 non-null  float64       
 14  year           29783 non-null  int32         
dtypes: datetime64[ns](1

In [13]:
#Se convierte a CSV el dataframe
data_df = df_games.to_csv(r"C:\Users\jotad\OneDrive\Documentos\Sebastian\PROYECTO\steam_games_final.csv", index=False) 